<a href="https://colab.research.google.com/github/thebishorup/nlp/blob/master/distil_bert_sentiment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ktrain

     |████████████████████████████████| 25.3MB 129kB/s 
     |████████████████████████████████| 983kB 49.0MB/s 
     |████████████████████████████████| 245kB 51.7MB/s 
     |████████████████████████████████| 890kB 49.7MB/s 
     |████████████████████████████████| 471kB 53.1MB/s 
     |████████████████████████████████| 3.0MB 52.6MB/s 
     |████████████████████████████████| 890kB 49.4MB/s 
     |████████████████████████████████| 1.1MB 45.1MB/s 
  Created wheel for ktrain: filename=ktrain-0.21.2-cp36-none-any.whl size=25269840 sha256=2f5f270236bd786b01c90f1f391993d55de946440c9f924628eabedf77219d5a
  Stored in directory: /root/.cache/pip/wheels/5a/16/d5/4673e3d2507440040ce1a144db9946b77d08a666e3dd25ddbc
  Created wheel for keras-bert: filename=keras_bert-0.86.0-cp36-none-any.whl size=34145 sha256=05e158a51f97e921f377462d6951196bc1ceaea3708f2f20c6e619548018c66c
  Stored in directory: /root/.cache/pip/wheels/66/f0/b1/748128b58562fc9e31b907bb5e2ab6a35eb37695e83911236b
  Created wheel for lan

In [2]:
!git clone https://github.com/laxmimerit/IMDB-Movie-Reviews-Large-Dataset-50k.git

Cloning into 'IMDB-Movie-Reviews-Large-Dataset-50k'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 10 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [3]:
import pandas as pd
import numpy as np
import ktrain
from ktrain import text
import tensorflow as tf

### LOAD TRAINING AND TEST DATASET

In [18]:
data_test = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/test.xlsx', dtype=str)
data_train = pd.read_excel('/content/IMDB-Movie-Reviews-Large-Dataset-50k/train.xlsx', dtype=str)

In [5]:
data_test.head()

,Reviews,Sentiment
0,Who would have thought that a movie about a ma...,pos
1,After realizing what is going on around us ......,pos
2,I grew up watching the original Disney Cindere...,neg
3,David Mamet wrote the screenplay and made his ...,pos
4,"Admittedly, I didn't have high expectations of...",neg


In [6]:
data_train.head()

,Reviews,Sentiment
0,"When I first tuned in on this morning news, I ...",neg
1,"Mere thoughts of ""Going Overboard"" (aka ""Babes...",neg
2,Why does this movie fall WELL below standards?...,neg
3,Wow and I thought that any Steven Segal movie ...,neg
4,"The story is seen before, but that does'n matt...",neg


In [7]:
data_test.shape, data_train.shape

((25000, 2), (25000, 2))

In [13]:
# COUNT THE SENTIMENT DATA DISTRIBUTION IN TEST AND TRAIN DATASET
data_train['Sentiment'].value_counts()

neg    12500
pos    12500
Name: Sentiment, dtype: int64

In [14]:
data_test['Sentiment'].value_counts()

neg    12500
pos    12500
Name: Sentiment, dtype: int64

In [16]:
text.print_text_classifiers()

fasttext: a fastText-like model [http://arxiv.org/pdf/1607.01759.pdf]
logreg: logistic regression using a trainable Embedding layer
nbsvm: NBSVM model [http://www.aclweb.org/anthology/P12-2018]
bigru: Bidirectional GRU with pretrained fasttext word vectors [https://fasttext.cc/docs/en/crawl-vectors.html]
standard_gru: simple 2-layer GRU with randomly initialized embeddings
bert: Bidirectional Encoder Representations from Transformers (BERT) [https://arxiv.org/abs/1810.04805]
distilbert: distilled, smaller, and faster BERT from Hugging Face [https://arxiv.org/abs/1910.01108]


In [19]:
(train, val, preproc) = text.texts_from_df(train_df=data_train, text_column='Reviews', 
                   label_columns='Sentiment', val_df=data_test, maxlen=512,
                   preprocess_mode='distilbert')

preprocessing train...
language: en
train sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 234
	95percentile : 598
	99percentile : 913


### BUILD THE MODEL

In [20]:
model = text.text_classifier(name = 'distilbert', train_data = train, preproc = preproc)

Is Multi-Label? False
maxlen is 512



done.


In [21]:
learner = ktrain.get_learner(model=model, train_data=train, val_data=val, batch_size=6)

### TRAIN THE LEARNER

In [22]:
learner.fit_onecycle(lr=2e-5, epochs=2)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/2
4167/4167 [==============================] - 1192s 286ms/step - loss: 0.2823 - accuracy: 0.8838 - val_loss: 0.1443 - val_accuracy: 0.9491
Epoch 2/2
4167/4167 [==============================] - 1188s 285ms/step - loss: 0.1526 - accuracy: 0.9444 - val_loss: 0.0586 - val_accuracy: 0.9844


### GET THE PREDICTOR

In [23]:
predictor = ktrain.get_predictor(model=learner.model, preproc=preproc)

### SAVINF THE PREDICTOR INTO 'GOOGLE DRIVE'

In [24]:
from google.colab import drive

In [25]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [26]:
predictor.save('/content/drive/My Drive/NLP_TRAINED_MODEL/distilbert')

### TESTING THE MODEL

In [27]:
data = ['This is complete waste of time and money. Not recommended', 'It could have been a better story but presentation was satisfactory.']

In [32]:
predictor.predict(data, return_proba=True)

/usr/local/lib/python3.6/dist-packages/ktrain/text/preprocessor.py:209: UserWarning: List or array of two texts supplied, so task being treated as text classification. If this is a sentence pair classification task, please cast to tuple.
  'If this is a sentence pair classification task, please cast to tuple.')


array([[0.9966068 , 0.00339316],
       [0.29901478, 0.7009852 ]], dtype=float32)

In [30]:
predictor.get_classes()

['neg', 'pos']